# ABOUT DATASET

Attribute information
1. business_id: business identification number of the companies in San Francisco
2. business_name: business name according of each business id.
3. business_address: Location of business
4. business_city: San Francisco.
5. business_state: California as 'CA'.
6. business_postal_code: 
7. business_latitude: 
8. business_longitude: 
9. business_location: 
10. business_phone_number: 
11. inspection_id: inspection identification number
12. inspection_score: the score of inspection (0-100)
13. inspection_type: 
14. violation_id: Computer generated id if there is violation after inspecting
15. violation_description: 
16. risk_category income: High, Low, Moderate, none


# DATA EXPLORATION

# Import libraries and load data

In [23]:
import pandas as pd
import numpy as np
q=pd.read_csv('sf_restaurant_health_violations.csv',encoding='latin-1')
pd.set_option('display.max_rows',500)

# Find all businesses which have low-risk safety violations

Find all businesses which have low-risk safety violations.

In [2]:
q[q.risk_category=='Low Risk']['violation_id'].count()

109

# Inspection Scores For Businesses
Find the median inspection score of each business and output the result along with the business name. Order records based on the inspection score in descending order.

In [3]:
q=q.groupby('business_name')['inspection_score'].mean().reset_index().sort_values(by='inspection_score',ascending=False).rename(columns={'inspection_score':'avg_inspection_score'})
q.head()

,business_name,avg_inspection_score
61,"Crepe and Brioche, Inc.",100.0
21,Bebebar Juice & Sandwich,100.0
144,MV Taurus,100.0
264,Yerba Buena Tea Co (formerly Tea Smiths of SF),100.0
141,MARTIN L. KING MIDDLE SCHOOL,100.0


# Find all business postal codes of restaurants with issues related to the water
Find all business postal codes of restaurants with issues related to the water (violation description contains substring "water").



In [4]:
# import the RegEx library for Regular Expressions
import re
q=pd.read_csv('sf_restaurant_health_violations.csv',encoding='latin-1')
# find the values cotain 'water' in the 'violation_description'
q.violation_description.str.contains('water',flags=re.I,regex=True,na=False)
# Find all business 
q[(q.violation_description.str.contains('water',flags=re.I,regex=True,na=False))].iloc[:,[1,5,-2]]

,business_name,business_postal_code,violation_description
165,S. F. Gourmet Hot Dog Cart,94133.0,Inadequate sewage or wastewater disposal
179,Gateway High/Kip Schools,94115.0,No hot water or running water
185,Iza Ramen,94103.0,Insufficient hot water or running water
210,PANCHO'S,94118.0,Insufficient hot water or running water
279,Da Cafe,94118.0,No hot water or running water


# Daily Violation Counts
Determine the change in the number of daily violations by calculating the difference between the count of current and previous violations by inspection date.
Output the inspection date and the change in the number of daily violations. Order your results by the earliest inspection date first.

In [5]:
q=pd.read_csv('sf_restaurant_health_violations.csv',encoding='latin-1')
# change the type of inspection_date column into datetime
q.inspection_date=q.inspection_date.apply(pd.to_datetime)
# Find the total violations of each inspection date, rename column & sort by the earliest inspection date
q=q.groupby('inspection_date')['violation_id'].count().reset_index().rename(columns={'violation_id':'total_violations'}).sort_values('inspection_date',ascending=False)
# find the previous violations of each inspection date, fill na with 0
q['previous_violations']=q.total_violations.shift(1).fillna(0)
# add the violation diff
q['violation_diff']=q.total_violations-q.previous_violations
q.head()

,inspection_date,total_violations,previous_violations,violation_diff
226,2018-08-28,1,0.0,1.0
225,2018-08-02,1,1.0,0.0
224,2018-07-25,0,1.0,-1.0
223,2018-07-24,1,0.0,1.0
222,2018-07-20,0,1.0,-1.0


# Verify that the first 4 digits are equal to 1415 for all phone numbers
Verify that the first 4 digits are equal to 1415 for all phone numbers.

In [24]:
# re-setup the dataframe
q=pd.read_csv('sf_restaurant_health_violations.csv',encoding='latin-1')
# change type of business_phone_number into string
q.business_phone_number=q.business_phone_number.astype(str)
# find the string with fist character equal 1415
q[q.business_phone_number.str.contains('^1415')].head().iloc[:,[0,1,2,9]]

,business_id,business_name,business_address,business_phone_number
6,79974,"Antonelli Brothers Meat, Fish, and Poultry Inc.",3585 California St,14155757413.0
13,87447,The Castro Republic,2295 Market St,14155930525.0
14,7747,SAFEWAY STORE #964,4950 Mission St,14155587200.0
20,87620,Dolores Park Outpost,698 Guerrero St,14155378648.0
23,80591,L & G Vietnamese Sandwich,602 Eddy St,14155886402.0


# Highest Number Of High-risk Violations
Find details of the business with the highest number of high-risk violations. Output all columns from the dataset considering business_id which consist 'high risk' phrase in risk_category column.

In [7]:
# re-setup the dataframe
q=pd.read_csv('sf_restaurant_health_violations.csv',encoding='latin-1')
# filter with high risk, find the total violations according to 'business_id', 'business_name', 'business_address', rename and sort
q[q.risk_category=='High Risk'].groupby(['business_id','business_name','business_address'])['violation_id'].count().reset_index().rename(columns={'violation_id':'total_violations'}).sort_values(by='total_violations',ascending=False).iloc[0:2,:]

,business_id,business_name,business_address,total_violations
30,86647,Da Cafe,407 Clement St,4
31,86718,Lollipot,890 Taraval St,2


# Number Of Inspections By Zip
Find the number of inspections that happened in the municipality with postal code 94102 during January, May or November in each year.
Output the count of each month separately.

In [8]:
# re-setup the dataframe
q=pd.read_csv('sf_restaurant_health_violations.csv',encoding='latin-1')
# change business postal code into string
q.business_postal_code=q.business_postal_code.astype(str)
# find the business whose postal code is 94102
q[q.business_postal_code.str.contains('94102')]
# change the data type of inspection date
q.inspection_date=q.inspection_date.apply(pd.to_datetime)
# find the inspections which happened during January, May or November
q[(q.inspection_date.dt.month == 1)|(q.inspection_date.dt.month == 5)|(q.inspection_date.dt.month == 11)]
# filter the dataset with 2 condition:
q=q[(q.business_postal_code.str.contains('94102'))&((q.inspection_date.dt.month == 1)|(q.inspection_date.dt.month == 5)|(q.inspection_date.dt.month == 11))]
# find the total violation of the aboved dataset
q.count()['violation_id']

5

# Inspections Per Risk Category
Count the number of inspections per each risk category.
Categorize records with null values under the 'No Risk' category.

Sort the result based on the number of inspections in descending order.

In [9]:
q=pd.read_csv('sf_restaurant_health_violations.csv',encoding='latin-1')
# fill the null values with the 'No Risk'
q.risk_category=q.risk_category.fillna('No Risk')
# groupby the total violations of each risk categories
q.groupby('risk_category')['violation_id'].count().reset_index().rename(columns={'violation_id':'total_violations'})

,risk_category,total_violations
0,High Risk,38
1,Low Risk,109
2,Moderate Risk,78
3,No Risk,0


# Find All Inspections Made On Restaurants
Find all inspections made on restaurants and output the business name and the inspection score. For this question business is considered as a restaurant if it contains string "restaurant" inside its name.

In [10]:
# import the RegEx library for Regular Expressions
import re
# reset the dataframe for new solution
q=pd.read_csv('sf_restaurant_health_violations.csv',encoding='latin-1')
# find all restaurants in dataset
q[q.business_name.str.contains('restaurant',flags=re.I,regex=True)]
# find all inspections made on restaurants
q[q.business_name.str.contains('restaurant',flags=re.I,regex=True)].violation_id.count()

34

# Number of violations
You're given a dataset of health inspections. Count the number of violation in an inspection in 'Roxanne Cafe' for each year. If an inspection resulted in a violation, there will be a value in the 'violation_id' column. Output the number of violations by year in ascending order.

In [11]:
# reset the dataframe for new solution
q=pd.read_csv('sf_restaurant_health_violations.csv',encoding='latin-1')
# change type of inspection date and get the year from it
q.inspection_date=q.inspection_date.apply(pd.to_datetime)
q['year']=q.inspection_date.dt.year
# filter out the 'Roxanne Cafe', find the total violations of each years, rename columns
q[q.business_name.str.contains('Roxanne Cafe')].groupby('year')['violation_id'].count().reset_index().rename(columns={'violation_id':'total_violations'})

,year,total_violations
0,2015,5
1,2016,2
2,2018,3


# Find the number of violations that each school had
Find the number of violations that each school had. Any inspection is considered a violation if its risk category is not null.
Output the corresponding business name along with the result.
Order the result based on the number of violations in descending order.

In [12]:
# reset the dataframe for new solution
q=pd.read_csv('sf_restaurant_health_violations.csv',encoding='latin-1')
# find the total violation of each school and risk category
q[q.business_name.str.contains('School|SCHOOL|school')].groupby('business_name')['risk_category'].value_counts()

business_name                risk_category
BALBOA HIGH SCHOOL           Low Risk         1
DENMAN MIDDLE SCHOOL         Low Risk         1
Francisco Middle School      Low Risk         1
Gateway High/Kip Schools     High Risk        1
John Chin Elementary School  Low Risk         1
Name: risk_category, dtype: int64

# Find the postal code which has the highest average inspection score
Find the postal code which has the highest average inspection score.
Output the corresponding postal code along with the result.

In [13]:
# reset data
q=pd.read_csv('sf_restaurant_health_violations.csv',encoding='latin-1')
# Substring of an entire 'business_postal_code' column
q.business_postal_code=q.business_postal_code.astype(str).str[:5]
# final result
q.groupby('business_postal_code')['inspection_score'].mean().round(0).reset_index().rename(columns={'inspection_score':'avg_inspection_score'}).sort_values(by='avg_inspection_score',ascending=False).head()

,business_postal_code,avg_inspection_score
4,94107,92.0
21,94132,92.0
19,94127,92.0
13,94117,91.0
24,nan,90.0


# Find the business names that have inspection scores of less than 50
Find the business names that scored less than 50 in inspections.
Output the result along with the corresponding inspection date and the score.

In [14]:
# reset the dataframe for new solution
q=pd.read_csv('sf_restaurant_health_violations.csv',encoding='latin-1')
q[q['inspection_score']<50].iloc[:,[0,1,-5,-6]]

,business_id,business_name,inspection_score,inspection_date
277,86718,Lollipot,45.0,5/22/2018
278,86718,Lollipot,45.0,5/22/2018
279,86647,Da Cafe,48.0,9/7/2016
280,86718,Lollipot,45.0,5/22/2018
281,86718,Lollipot,45.0,5/22/2018
282,86718,Lollipot,45.0,5/22/2018
283,86718,Lollipot,45.0,5/22/2018
284,86647,Da Cafe,48.0,9/7/2016
285,86647,Da Cafe,48.0,9/7/2016
286,86647,Da Cafe,48.0,9/7/2016


# Find the first and last times the maximum score was awarded
Find the first and last times the maximum score was awarded



In [25]:
# reset the dataframe for new solution
q=pd.read_csv('sf_restaurant_health_violations.csv',encoding='latin-1')
# change the datatype of 'inspection date' column
q.inspection_date=q.inspection_date.apply(pd.to_datetime)
# find the data which equal the maximum inspection score, sort value by date, find the first and the last of them
q[q.inspection_score==q.inspection_score.max()].sort_values(by='inspection_date',ascending=False).iloc[[0,-1],[0,1,2,3,-5,-6]]

,business_id,business_name,business_address,business_city,inspection_score,inspection_date
131,70755,North Point Market,3098 Polk St,San Francisco,100.0,2018-06-15
17,5862,MARTIN L. KING MIDDLE SCHOOL,350 GIRARD St,San Francisco,100.0,2015-09-23


# Find the business names that have inspection scores of less than 50
Find the business names that scored less than 50 in inspections.
Output the result along with the corresponding inspection date and the score.

In [16]:
# reset the dataframe for new solution
q=pd.read_csv('sf_restaurant_health_violations.csv',encoding='latin-1')
q[q.inspection_score<50].iloc[:,[0,1,-6,-5]]

,business_id,business_name,inspection_date,inspection_score
277,86718,Lollipot,5/22/2018,45.0
278,86718,Lollipot,5/22/2018,45.0
279,86647,Da Cafe,9/7/2016,48.0
280,86718,Lollipot,5/22/2018,45.0
281,86718,Lollipot,5/22/2018,45.0
282,86718,Lollipot,5/22/2018,45.0
283,86718,Lollipot,5/22/2018,45.0
284,86647,Da Cafe,9/7/2016,48.0
285,86647,Da Cafe,9/7/2016,48.0
286,86647,Da Cafe,9/7/2016,48.0


# Find the average of inspections scores between 91 and 100
Find the mean of inspections scores between 91 and 100.
Assuming that the scores are normally distributed.

In [17]:
# reset the dataframe for new solution
q=pd.read_csv('sf_restaurant_health_violations.csv',encoding='latin-1')
# final results:
q[(q.inspection_score<=100)&(q.inspection_score>=91)].iloc[:,[0,1,-5]].head()

,business_id,business_name,inspection_score
0,5800,John Chin Elementary School,98.0
6,79974,"Antonelli Brothers Meat, Fish, and Poultry Inc.",94.0
7,1939,STARBUCKS COFFEE CO. #603,98.0
17,5862,MARTIN L. KING MIDDLE SCHOOL,100.0
26,86780,Peet's Coffee & Tea,96.0


# Classify Business Type
Classify each business as either a restaurant, cafe, school, or other.


•	A restaurant should have the word 'restaurant' in the business name.
•	A cafe should have either 'cafe', 'café', or 'coffee' in the business name.
•	A school should have the word 'school' in the business name.
•	All other businesses should be classified as 'other'.


Output the business name and their classification.

In [18]:
# import the RegEx library for Regular Expressions
import re
# reset the dataframe for new solution
q=pd.read_csv('sf_restaurant_health_violations.csv',encoding='latin-1')

In [19]:
# Condition 1: All restaurants in dataset: q.business_name.str.contains('restaurant',flags=re.I,regex=True)
# Condition 2: All cafe in dataset: q.business_name.str.contains('cafe|coffee|café',flags=re.I,regex=True)
# Condition 3: All schools in dataset: q.business_name.str.contains('school',flags=re.I,regex=True)

In [20]:
# find the other and fill 'other' into the 'classification'
q['classification']='other'
# fill 'restaurant' with the condition 1
q.loc[q.business_name.str.contains('restaurant',flags=re.I,regex=True),'classification']= 'restaurant'
# fill 'cafe' with the condition 2
q.loc[q.business_name.str.contains('cafe|coffee|café',flags=re.I,regex=True),'classification']= 'cafe'
# fill 'school' with the condition 3
q.loc[q.business_name.str.contains('school',flags=re.I,regex=True),'classification']= 'school'
#  final resul
q.iloc[:,[1,-1]]
# the total businesses of each classifications
q.groupby('classification')['business_id'].count()

classification
cafe           50
other         202
restaurant     38
school          7
Name: business_id, dtype: int64

# Find all businesses whose lowest and highest inspection scores are different
Find all businesses whose lowest and highest inspection scores are different.
Output the corresponding business name and the lowest and highest scores of each business.
Order the result based on the business name in ascending order.

In [21]:
q=pd.read_csv('sf_restaurant_health_violations.csv',encoding='latin-1')
# Fill the missing values of inspection score with zero
q.inspection_score=q.inspection_score.fillna(0)
# Create a highest inspection score of each bussinesses
q['highest_inspection_score']=q.groupby('business_name')['inspection_score'].transform(max)
# Create a lowest inspection score of each bussinesses
q['lowest_inspection_score']=q.groupby('business_name')['inspection_score'].transform(min)
# Find all businesses whose lowest and highest inspection scores are different.
q[q['highest_inspection_score']!=q['lowest_inspection_score']].iloc[:,[0,1,-1,-2]].sort_values(by='business_name',ascending=True)
q=q[q['highest_inspection_score']!=q['lowest_inspection_score']].iloc[:,[0,1,-1,-2]].sort_values(by='business_name',ascending=True)
# Find the unique values of table
q[~q.duplicated()]

,business_id,business_name,lowest_inspection_score,highest_inspection_score
74,83351,City Super,77.0,78.0
264,89276,"Fresca Gardens, Inc",82.0,88.0
8,68872,Jiang Ling Cuisine Restaurant,72.0,74.0
254,86780,Peet's Coffee & Tea,0.0,96.0
177,83046,Project Juice,96.0,100.0
212,80302,Project Juice,96.0,100.0
288,500,Roxanne Cafe,86.0,92.0
58,7747,SAFEWAY STORE #964,0.0,81.0
271,64236,Sutter Pub and Restaurant,88.0,90.0
128,90839,The Grove - Design District,0.0,96.0


# Worst Businesses


For every year, find the worst business in the dataset. The worst business has the most violations during the year. You should output the year, business name, and number of violations.



In [22]:
q=pd.read_csv('sf_restaurant_health_violations.csv',encoding='latin-1')
# change data type of inspection date into datetime
q.inspection_date=q.inspection_date.apply(pd.to_datetime)
q['year']=q.inspection_date.dt.year
# find the total viloation per business_name of each years
q['total_violations']=q.groupby(['year','business_name'])['violation_id'].transform('count')
# add the column of max violations of each year
qq=q.groupby(['year'])['total_violations'].max().reset_index()
# merge the table which contains the total violations with the table which contains the max violations of each year
q=pd.merge(q,qq,left_on=['year','total_violations'],right_on=['year','total_violations'],how='inner')
# remove duplicated
q[~q.duplicated(subset=['business_id','year'])]
q=q[~q.duplicated(subset=['business_id','year'])]
# sort out the nessary columns and sort values by year in descending
q.iloc[:,[-2,-1,0,1]].sort_values(by='year',ascending=False)

,year,total_violations,business_id,business_name
3,2018,6,86718,Lollipot
0,2017,2,86780,Peet's Coffee & Tea
9,2016,4,86647,Da Cafe
13,2015,5,500,Roxanne Cafe
